### Step 1 Create a dataframe by scraping a web page

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium


#import data from a web page and scrape the table
res=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=BeautifulSoup(res.content,'html.parser')
table=soup.find_all('table')[0]
canada_df=pd.read_html(str(table))[0]

Solving environment: done

# All requested packages already installed.



In [43]:
#remove 'not assigned' in column 'Borough'
canada_df.replace('Not assigned',np.nan,inplace=True)
canada_df.dropna(subset=['Borough'],axis=0,inplace=True)
#replace 'not assigned' in column 'Neighbourhood' with the corresponding borough
canada_df.fillna(method='ffill',axis=1,inplace=True)
#group neighbourhoods in the same borough and create a clean dataframe can_df
can_df=canada_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [44]:
can_df.shape

(103, 3)

### Step 2 Add latitude and longitude coordinates to each postcode

In [45]:
#geocoder cannot retrieve the coordinates so the csv file is used
coord_df=pd.read_csv('http://cocl.us/Geospatial_data')
coord_df.rename(columns={'Postal Code':'Postcode'},inplace=True)
#combine two dataframes as a new dataframe
data_df=pd.merge(can_df,coord_df,on=['Postcode'])

In [46]:
#explore neighborhoods in Toronto
toronto_df=data_df[data_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


### Step 3 Explore and cluster boroughs

In [47]:
#Hidden cell
CLIENT_ID = 'K1QQ31GDDLSAXRJ2RCDI134XGVE5G0WNA0H1XW3TZOXV4VGS'
CLIENT_SECRET = 'SO3ADVRZ540GGKK5VI3O4VDTLAXBHDEDLXX0JQXX0KNKTEM5'
VERSION = '20191018'

In [48]:
#define a function to clean the category data
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
#fetch venue data from Foursquare for each postcode
venues=pd.DataFrame()

for i in range(len(toronto_df)):
    LATITUDE=toronto_df['Latitude'][i]
    LONGITUDE=toronto_df['Longitude'][i]
    url='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,LATITUDE,LONGITUDE,500,100)
    results=requests.get(url).json()
    venues_temp=results['response']['venues']
    venues_temp=json_normalize(venues_temp)
    filtered_columns=['name','location.postalCode','location.lat','location.lng','categories']
    venues_temp=venues_temp.loc[:,filtered_columns]
    venues_temp.dropna(subset=['location.postalCode'],axis=0,inplace=True)
    venues_temp['location.postalCode']=venues_temp['location.postalCode'].astype(str).str[0:3].str.upper()
    venues_temp['categories']=venues_temp.apply(get_category_type,axis=1)
    venues=pd.concat([venues,venues_temp],axis=0,sort=False,ignore_index=True)

venues.rename(columns={'name':'Name','location.postalCode':'Postcode','location.lat':'Venue_Latitude','location.lng':'Venue_Longitude','categories':'Categories'},inplace=True)    
data_df=pd.merge(venues,toronto_df,on=['Postcode'])

In [50]:
data_df

,Name,Postcode,Venue_Latitude,Venue_Longitude,Categories,Borough,Neighbourhood,Latitude,Longitude
0,Beaches Fitness - Personal Trainer & Health Coach,M4L,43.669253,-79.311140,Gym / Fitness Center,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
1,British Style Fish & Chips,M4L,43.668723,-79.317139,Fish & Chips Shop,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
2,Zipcar,M4L,43.667100,-79.314800,Rental Car Location,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,TTC Russell Substation,M4L,43.666293,-79.315828,Power Plant,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
4,Harvey's,M4L,43.666663,-79.315081,Fast Food Restaurant,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
5,Off Centre DJ School,M4L,43.671670,-79.318887,College Classroom,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
6,Paint Nite,M4L,43.667302,-79.312767,Nightlife Spot,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
7,Sixgreen Labs Inc.,M4L,43.669213,-79.319305,Tech Startup,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
8,Casa di Giorgio,M4L,43.666645,-79.315204,Italian Restaurant,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
9,Village Hardware,M4L,43.666257,-79.317169,Hardware Store,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572


In [51]:
#one hot encoding
toronto_onehot=pd.get_dummies(data_df[['Categories']], prefix="", prefix_sep="")
toronto_onehot['Postcode']=data_df['Postcode'] 
toronto_grouped=toronto_onehot.groupby('Postcode').mean().reset_index()

In [52]:
#cluster neiborhoods
k=5
toronto_clustering=toronto_grouped.drop(['Postcode'],axis=1)
kmeans=KMeans(n_clusters=k,random_state=0)
kmeans.fit(toronto_clustering)
labels=kmeans.labels_
toronto_grouped['Labels']=labels
toronto_clustered=toronto_grouped[['Postcode','Labels']]
toronto_clustered_2=pd.merge(toronto_clustered,toronto_df,on=['Postcode'])
toronto_clustered_2

,Postcode,Labels,Borough,Neighbourhood,Latitude,Longitude
0,M4E,0,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,0,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,0,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,0,East Toronto,Studio District,43.659526,-79.340923
4,M4N,0,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,0,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,2,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,0,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,3,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,4,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [53]:
#create a map
map_clusters=folium.Map(location=[43.653963,-79.387207],zoom_start=11)
x=np.arange(k)
ys=[i+x+(i*x)**2 for i in range(k)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clustered_2['Latitude'],toronto_clustered_2['Longitude'],toronto_clustered_2['Neighbourhood'],toronto_clustered_2['Labels']):
    label=folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters